In [1]:
import sys
from pathlib import Path

import pandas as pd

sys.path.insert(0, '/home/tas/Workspace/pysimple/src')
from pysimple.io import to_tsv, from_tsv

In [2]:
DATA_DIR = Path('/mnt/storage/tas/data/xeno-canto')

## Birds in Kyiv

Bird species in Kyiv: http://www.dom-prirody.com.ua/priroda-kieva/ptahi

In [3]:
birds = pd.DataFrame(columns=['area', 'name', 'gen', 'sp'], data=[
    ('city', 'Боривітер звичайний,Обыкнове́нная пустельга́', 'falco', 'tinnunculus'),
    ('city', 'Підсоколик великий,Чеглок', 'falco', 'subbuteo'),
    ('city', 'Серпокрилець звичайний,Стриж чорний', 'apus', 'apus'),
    ('city', 'Дятел звичайний,Большой пёстрый дятел', 'dendrocopos', 'major'),
    ('city', 'Дятел сирійський', 'dendrocopos', 'syriacus'),
    ('city', 'Ластівка міська,Городская ласточка', 'delichon', 'urbicum'),
    ('city', 'Шпак звичайний,Обыкнове́нный скворе́ц', 'sturnus', 'vulgaris'),
    ('city', 'Мухоловка сіра', 'muscicapa', 'striata'),
    ('city', 'Мухоловка мала', 'ficedula', 'parva'),
    ('city', 'Дрізд чорний', 'turdus', 'merula'),
    ('city', 'Горихвістка чорна,Горихво́стка-черну́шка', 'phoenicurus', 'ochruros'),
    ('city', 'Зеленяк,Обыкнове́нная зелену́шка', 'chloris', 'chloris'),
    ('city', 'Зяблик', 'fringilla', 'coelebs'),
    ('city', 'Горобець хатній,Домо́вый воробе́й', 'passer', 'domesticus'),
    ('city', 'Плиска біла,Белая трясогузка', 'motacilla', 'alba'),
    ('city', 'Синиця велика', 'parus', 'major'),
    ('city', 'Ворона сіра', 'corvus', 'cornix'),
    ('city', 'Сорока', 'pica', 'pica'),
    ('park', 'Сова вухата,Уша́стая сова́', 'asio', 'otus'),
    ('park', 'Сова сіра', 'strix', 'aluco'),
    ('park', 'Дятел середній', 'dendrocoptes', 'medius'),
    ('park', 'Дятел малий,Малый пёстрый дятел', 'dryobates', 'minor'),
    ('park', 'Крутиголовка,Вертише́йка', 'jynx', 'torquilla'),
    ('park', 'Мухоловка-строката,Мухоло́вка-пестру́шка,Березовка', 'ficedula', 'hypoleuca'),
    ('park', 'Кропи́в\'янка сі́ра,Се́рая сла́вка', 'sylvia', 'communis'),
    ('park', 'Кропив\'янка садова,Садовая славка', 'sylvia', 'borin'),
    ('park', 'Кропив\'янка чорноголова,Черноголовая славка,Черноголовка', 'sylvia', 'atricapilla'),
    ('park', 'Кропив\'янка прудка,Славка-завирушка,Cлавка-мельничек', 'sylvia', 'curruca'),
    ('park', 'Берестянка звичайна,Зелёная пересмешка', 'hippolais', 'icterina'),
    ('park', 'Вівчарник жовтобровий,Пеночка-трещотка,Березовка', 'phylloscopus', 'sibilatrix'),
    ('park', 'Соловейко східний', 'luscinia', 'luscinia'),
    ('park', 'Дрізд-чикотень,Ряби́нник,Дрозд-рябинник', 'turdus', 'pilaris'),
    ('park', 'Вільша́нка,Заря́нка', 'erithacus', 'rubecula'),
    ('park', 'Ка́м\'янка звича́йна,Ка́менка', 'oenanthe', 'oenanthe'),
    ('park', 'Щедрик,Канареечный вьюрок,Желтозобик', 'serinus', 'serinus'),
    ('park', 'Щиглик,Черноголовый щегол', 'carduelis', 'carduelis'),
    ('park', 'Коноплянка,Реполо́в', 'linaria', 'cannabina'),
    ('park', 'Повзик,Обыкновенный по́ползень,Ямщи́к', 'sitta', 'europaea'),
    ('park', 'Підкори́шник звича́йний,Обыкновенная пищуха', 'certhia', 'familiaris'),
    ('park', 'Синиця блакитна,Обыкнове́нная лазо́ревка,Зелёная лазо́ревка', 'cyanistes', 'caeruleus'),
    ('park', 'Сойка,Кареза', 'garrulus', 'glandarius'),
    ('park', 'Крук,ворон,кракун', 'corvus', 'corax')
])

In [4]:
birds_path = DATA_DIR / 'birds' / 'birds.tsv'
to_tsv(filepath=birds_path, data=birds)

In [5]:
birds_path = DATA_DIR / 'birds' / 'birds.tsv'
birds = from_tsv(birds_path)
birds['gen_sp'] = birds['gen'] + ' ' + birds['sp']

In [6]:
# Load existing records for selected species
records_path = DATA_DIR / 'records' / 'records.tsv'
records = from_tsv(records_path, usecols='gen,sp,q'.split(','))
records['gen_sp'] = records['gen'].str.lower() + ' ' + records['sp'].str.lower()
records = records[records['gen_sp'].isin(birds['gen_sp'])]

### Dataset size

In [7]:
birds['size'] = birds['gen_sp'].map(records['gen_sp'].value_counts())

### Calls quality

In [8]:
records['quality'] = records['q'].map(dict(A=5, B=4, C=3, D=2, E=1, no_score=-1))
birds['quality'] = birds['gen_sp'].map(records[records['quality'].gt(0)].groupby('gen_sp')['quality'].mean())

In [9]:
birds.sort_values('size', ascending=False)[['name', 'gen_sp', 'size', 'quality']]

,name,gen_sp,size,quality
15,Синиця велика,parus major,4669,3.608267
12,Зяблик,fringilla coelebs,3430,3.760481
9,Дрізд чорний,turdus merula,3363,3.730735
32,"Вільша́нка,Заря́нка",erithacus rubecula,2413,3.795645
39,"Синиця блакитна,Обыкнове́нная лазо́ревка,Зелён...",cyanistes caeruleus,2368,3.484349
26,"Кропив'янка чорноголова,Черноголовая славка,Че...",sylvia atricapilla,2330,3.901505
24,"Кропи́в'янка сі́ра,Се́рая сла́вка",sylvia communis,1709,3.762305
13,"Горобець хатній,Домо́вый воробе́й",passer domesticus,1291,3.568979
3,"Дятел звичайний,Большой пёстрый дятел",dendrocopos major,1270,4.114195
19,Сова сіра,strix aluco,1092,3.677360
